# Export workflow to Neurodata Without Borders file and upload to DANDI

## Setup

First, let's change directories to find the `dj_local_conf` file.

In [ ]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')

In [ ]:
# We'll be working with long tables, so we'll make visualization easier with a limit
import datajoint as dj
dj.config['display.limit']=10

CodeBook users should also set a couple additional config parameters.

In [ ]:
username_as_prefix = dj.config["database.user"] + "_"
if not dj.config['custom']:
    dj.config['custom'] = {}
dj.config['custom'].update({
        "ephys_mode": "no-curation",
        "database.prefix": username_as_prefix,
        })

If you haven't already populated the `lab`, `subject`, `session`, `probe`, and `ephys` schemas, please do so now with [04-automate](./04-automate-optional.ipynb). Note: exporting `ephys` data is currently only supported on the `no_curation` schema. 

In [2]:
from workflow_array_ephys.pipeline import lab, subject, session, probe, ephys
from workflow_array_ephys.export import (element_lab_to_nwb_dict, subject_to_nwb, 
                                         session_to_nwb, ecephys_session_to_nwb, 
                                         write_nwb)
from element_interface.dandi import upload_to_dandi

Connecting cbroz@dss-db.datajoint.io:3306


## Export to NWB

We'll use the following keys to demonstrate export functions.

In [3]:
lab_key={"lab": "LabA"}
protocol_key={"protocol": "ProtA"}
project_key={"project": "ProjA"}
session_key={"subject": "subject5",
             "session_datetime": "2018-07-03 20:32:28"}

### Upstream Elements

If you plan to use all upstream Elements, you can skip to the following section. To combine with other schemas, the following functions may be helpful.

- **Element Lab** `element_lab_to_nwb_dict` exports NWB-relevant items to `dict` format.
- **Element Animal** `subject_to_nwb` returns an NWB file with subject information.
- **Element Session** `session_to_nwb` returns an NWB file with subject and session information.

Note: `pynwb` will display a warning regarding timezone information - datetime fields are assumed to be in local time, and will be converted to UTC.


In [ ]:
## If you don't already have data in the Element Lab
lab.Lab.insert1(
    {
        **lab_key,
        "lab_name": "LabA",
        "institution": "",
        "address": "",
        "time_zone": "UTC+0",
    },
    skip_duplicates=True,
)
lab.ProtocolType.insert1({"protocol_type": "A"})
lab.Protocol.insert1({**protocol_key, "protocol_type": "A"}, skip_duplicates=True)
lab.Project.insert1(project_key, skip_duplicates=True)

In [ ]:
print('Lab:\n')
print(element_lab_to_nwb_dict(lab_key=lab_key, protocol_key=protocol_key, 
                              project_key=project_key))
print('\nAnimal:\n')
print(subject_to_nwb(session_key=session_key))
print('\nSession:\n')
print(session_to_nwb(session_key=session_key))

### Element Array Electrophysiology

`ecephys_session_to_nwb` provides a full export mechanism, returning an NWB file with raw data, spikes, and LFP. Optional arguments determine which pieces are exported. For demonstration purposes, we recommend limiting `end_frame`.


In [22]:
help(ecephys_session_to_nwb)

Help on function ecephys_session_to_nwb in module element_array_ephys.export.nwb.nwb:

ecephys_session_to_nwb(session_key, raw=True, spikes=True, lfp='source', end_frame=None, lab_key=None, project_key=None, protocol_key=None, nwbfile_kwargs=None)
    Main function for converting ephys data to NWB
    
    Parameters
    ----------
    session_key: dict
    raw: bool
        Whether to include the raw data from source. SpikeGLX and OpenEphys are supported
    spikes: bool
        Whether to include CuratedClustering
    lfp:
        "dj" - read LFP data from ephys.LFP
        "source" - read LFP data from source (SpikeGLX supported)
        False - do not convert LFP
    end_frame: int, optional
        Used to create small test conversions where large datasets are truncated.
    lab_key, project_key, and protocol_key: dictionaries used to look up optional additional metadata
    nwbfile_kwargs: dict, optional
        - If element-session is not being used, this argument is required an

In [4]:
nwbfile = ecephys_session_to_nwb(session_key=session_key,
                                 raw=True,
                                 spikes=True,
                                 lfp="dj",
                                 end_frame=100,
                                 lab_key=lab_key,
                                 project_key=project_key,
                                 protocol_key=protocol_key,
                                 nwbfile_kwargs=None)

/Users/cb/miniconda3/envs/ele/lib/python3.8/site-packages/pynwb/file.py:1037: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
creating units table for paramset 0: 100%|██████████| 499/499 [00:41<00:00, 12.11it/s]


In [5]:
nwbfile

root pynwb.file.NWBFile at 0x140297891486016
Fields:
  acquisition: {
    ElectricalSeries1 <class 'pynwb.ecephys.ElectricalSeries'>,
    ElectricalSeries2 <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    262716621 <class 'pynwb.device.Device'>,
    714000838 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    probe262716621_shank0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    probe714000838_shank0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Example project to populate element-lab
  experimenter: ['User1']
  file_create_date: [datetime.datetime(2022, 5, 31, 15, 47, 41, 270996, tzinfo=tzlocal())]
  identifier: 172f2d3b-44c1-4ae1-8785-2d20d3df3db1
  institution: Example Uni
  keywords: ['Example' 'Study']
  lab: The Example Lab
  notes: Protocol for managing data ingestion
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  protocol: ProtA
  related_publicatio

`write_nwb` can then be used to write this file to disk. The following cell will include a timestamp in the filename.

In [ ]:
import time
my_path = "./"
my_path = f"/home/{dj.config['database.user']}/" # for codebook users
write_nwb(nwbfile, my_path+time.strftime("_test_%Y%m%d-%H%M%S.nwb"))

## DANDI Upload

`element-interface.dandi` includes the `upload_to_dandi` utility to support direct uploads. For more information, see [DANDI documentation](https://www.dandiarchive.org/handbook/10_using_dandi/).

In order to upload, you'll need...
1. A DANDI account
2. A `DANDI_API_KEY`
3. A `dandiset_id`

These values can be added to your `dj.config` as follows:

In [ ]:
dj.config['custom']['dandiset_id']="<six digits as string>" 
dj.config['custom']['dandi.api']="<40-character alphanumeric string>"

This would facilitate routine updating of your dandiset.

In [5]:
upload_to_dandi(
    data_directory="./temp_nwb/",
    dandiset_id=dj.config['custom']['dandiset_id'],
    staging=True,
    working_directory="./temp_nwb/",
    api_key=dj.config['custom']['dandi.api'],
    sync=False)

PATH                 SIZE     DONE            DONE% CHECKSUM STATUS          MESSAGE   
dandiset.yaml                                                done            updated   
test1.nwb            109.8 MB 109.8 MB         100%    ok    done                      
Summary:             109.8 MB 109.8 MB                       2 done          1 updated 
                              100.00%                                                  
Usage: dandi [OPTIONS] COMMAND [ARGS]...

  A client to support interactions with DANDI archive
  (http://dandiarchive.org).

  To see help for a specific command, run

      dandi COMMAND --help

  e.g. dandi upload --help

Options:
  --version
  -l, --log-level [DEBUG|INFO|WARNING|ERROR|CRITICAL]
                                  Log level (case insensitive).  May be
                                  specified as an integer.  [default: INFO]
  --pdb                           Fall into pdb if errors out
  --help                          Show this messag

pynwb validation errors for /Users/cb/Documents/dev/workflow-array-ephys/temp_nwb/200178/test1.nwb: []


PATH                 SIZE     ERRORS UPLOAD STATUS           MESSAGE                  
test1.nwb            109.8 MB   0           skipped          file exists              
dandiset.yaml        2.0 kB                 skipped          should be edited online  
Summary:             109.8 MB               2 skipped        1 file exists            
                                                             1 should be edited online
